

# **Atividade 5**

*   Cristiano Moreira Martins
*   Data: 03/04/2024




# **Exemplo de Anotação Linguistica**

In [56]:
#https://www.kaggle.com/code/italomarcelo/nlp-spacy-portuguese

import os
import chardet
import pandas as pd

import spacy.cli
from pathlib import Path
from collections import Counter

In [45]:

#modelos de linguagem para o portugues: https://spacy.io/models/pt
#spacy.cli.download("pt_core_news_sm")
#spacy.cli.download("pt_core_news_md")
spacy.cli.download("pt_core_news_lg")

#Carregando o modelo e o texto
#nlp_sm = spacy.load("pt_core_news_sm")
nlp_lg = spacy.load("pt_core_news_lg")
#nlp = spacy.load("pt_core_news_lg")
#texto = "PUCRS é a melhor universidade privada da região sul.\nCarlos estuda na Politécnica.\nCarlos foi com Maria ao Prédio 32 do campus."
#doc = nlp(texto)
#print(doc.text)

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
PUCRS é a melhor universidade privada da região sul.
Carlos estuda na Politécnica.
Carlos foi com Maria ao Prédio 32 do campus.


#### Carrega o dataset

In [41]:
def read_files(path):
  texts = pd.DataFrame()
  for filename in os.listdir(path):

    file_path = path + filename
    encoding = 'utf-8'
    with open(file_path, 'rb') as file:
      raw_data = file.read()
      result = chardet.detect(raw_data)
      encoding = result['encoding']

    with open(file_path, mode='r', encoding=encoding) as file:
      lines = file.readlines()

      tmp_data = pd.DataFrame()
      tmp_data['line'] = lines
      tmp_data['filename'] = filename
      texts = pd.concat([texts, tmp_data])

  texts.reset_index(inplace=True)

  return texts

In [42]:
root_path = './esportes/'
texts = read_files(root_path)
texts

,index,line,filename
0,0,"O anúncio oficial deve ocorrer nesta semana, p...",ESPORTE_2005_21964.txt
1,1,"O novo time de Massa é a Ferrari, que havia pl...",ESPORTE_2005_21964.txt
2,2,Mas as evidências começam a aparecer. Além das...,ESPORTE_2005_21964.txt
3,3,"À tarde, deixou escapar para um jornalista ale...",ESPORTE_2005_21964.txt
4,0,"O ala-pivô Nenê, 22, maior astro do basquete n...",ESPORTE_2005_22023.txt
...,...,...,...
70,3,"Se o Co-Rio não conseguir, a prefeitura arcará...",ESPORTE_2005_21873.txt
71,4,"Hoje, o município já é responsável por R$ 663 ...",ESPORTE_2005_21873.txt
72,5,"O contrato, no qual a prefeitura só será respo...",ESPORTE_2005_21873.txt
73,6,"Ruy Cézar, da secretaria municipal extraordiná...",ESPORTE_2005_21873.txt


#### 1. Identifique os substantivos existentes no dataset dado.

In [50]:
doc = nlp_lg(texts['line'][0])

for token in doc:
  if token.pos_ == 'NOUN':
    print((token.text, token.lemma_, token.pos_))

('anúncio', 'anúncio', 'NOUN')
('semana', 'semana', 'NOUN')
('saída', 'saída', 'NOUN')
('contrato', 'contrato', 'NOUN')
('chefe', 'chefe', 'NOUN')
('escuderia', 'escuderia', 'NOUN')


In [61]:
for idx, r in texts.iterrows():

  doc = nlp_lg(r['line'])

  nouns = []
  for token in doc:
    if token.pos_ == 'NOUN':
      nouns.append(token.text)

  texts.at[idx, 'NOUNS'] = ','.join(nouns)

texts

,index,line,filename,NOUNS
0,0,"O anúncio oficial deve ocorrer nesta semana, p...",ESPORTE_2005_21964.txt,"anúncio,semana,saída,contrato,chefe,escuderia"
1,1,"O novo time de Massa é a Ferrari, que havia pl...",ESPORTE_2005_21964.txt,"time,Massa,contratação,evento,termos,acordo,co..."
2,2,Mas as evidências começam a aparecer. Além das...,ESPORTE_2005_21964.txt,"evidências,frases,dirigente,excitação,domingo,..."
3,3,"À tarde, deixou escapar para um jornalista ale...",ESPORTE_2005_21964.txt,"tarde,jornalista,fluência,italiano,entrevista,..."
4,0,"O ala-pivô Nenê, 22, maior astro do basquete n...",ESPORTE_2005_22023.txt,"ala-pivô,astro,basquete,seleção,mudanças"
...,...,...,...,...
70,3,"Se o Co-Rio não conseguir, a prefeitura arcará...",ESPORTE_2005_21873.txt,"prefeitura,obra,compromisso,exigência,construç..."
71,4,"Hoje, o município já é responsável por R$ 663 ...",ESPORTE_2005_21873.txt,"município,investimento,evento"
72,5,"O contrato, no qual a prefeitura só será respo...",ESPORTE_2005_21873.txt,"contrato,prefeitura,débito,patrocínios,consórc..."
73,6,"Ruy Cézar, da secretaria municipal extraordiná...",ESPORTE_2005_21873.txt,"conta,município,prefeitura,simbologia,comitê"


#### 2. Usando o lema de cada substantivo, exiba os 20 mais frequentes.

In [63]:
lemmas = []
for idx, r in texts.iterrows():
  doc = nlp_lg(r['line'])

  for token in doc:
    if token.pos_ == 'NOUN':
      lemmas.append(token.lemma_)

counter = Counter(lemmas)

most_common = counter.most_common(20)

most_common

[('ano', 9),
 ('clube', 9),
 ('seleção', 8),
 ('cidade', 8),
 ('dia', 8),
 ('prefeitura', 8),
 ('jogo', 8),
 ('contrato', 6),
 ('time', 6),
 ('evento', 6),
 ('dirigente', 6),
 ('vez', 6),
 ('disputa', 6),
 ('onda', 6),
 ('equipe', 5),
 ('história', 5),
 ('sede', 5),
 ('jogador', 4),
 ('torneio', 4),
 ('competição', 4)]

#### 3. Identifique os verbos existentes no dataset dado.

In [64]:
for idx, r in texts.iterrows():

  doc = nlp_lg(r['line'])

  nouns = []
  for token in doc:
    if token.pos_ == 'VERB':
      nouns.append(token.text)

  texts.at[idx, 'VERBS'] = ','.join(nouns)

texts

,index,line,filename,NOUNS,VERBS
0,0,"O anúncio oficial deve ocorrer nesta semana, p...",ESPORTE_2005_21964.txt,"anúncio,semana,saída,contrato,chefe,escuderia","deve,ocorrer,admite,seguir,firmar,diz,ocorreu,..."
1,1,"O novo time de Massa é a Ferrari, que havia pl...",ESPORTE_2005_21964.txt,"time,Massa,contratação,evento,termos,acordo,co...","planejado,divulgar,revelaria,assinado,pode,mud..."
2,2,Mas as evidências começam a aparecer. Além das...,ESPORTE_2005_21964.txt,"evidências,frases,dirigente,excitação,domingo,...","começam,aparecer,conter,estava,largar,falado"
3,3,"À tarde, deixou escapar para um jornalista ale...",ESPORTE_2005_21964.txt,"tarde,jornalista,fluência,italiano,entrevista,...","deixou,escapar,precisa,melhorar,publicada,falo..."
4,0,"O ala-pivô Nenê, 22, maior astro do basquete n...",ESPORTE_2005_22023.txt,"ala-pivô,astro,basquete,seleção,mudanças","reiterou,defender,houver"
...,...,...,...,...,...
70,3,"Se o Co-Rio não conseguir, a prefeitura arcará...",ESPORTE_2005_21873.txt,"prefeitura,obra,compromisso,exigência,construç...","conseguir,arcará,financiar"
71,4,"Hoje, o município já é responsável por R$ 663 ...",ESPORTE_2005_21873.txt,"município,investimento,evento",
72,5,"O contrato, no qual a prefeitura só será respo...",ESPORTE_2005_21873.txt,"contrato,prefeitura,débito,patrocínios,consórc...","levantar,apresentado,viabilizou"
73,6,"Ruy Cézar, da secretaria municipal extraordiná...",ESPORTE_2005_21873.txt,"conta,município,prefeitura,simbologia,comitê","criada,organizar,admitiu,pode,ficar,sabe,vende..."


#### 4. Usando o lema de cada verbo, exiba os 20 mais frequentes.

In [65]:
lemmas = []
for idx, r in texts.iterrows():
  doc = nlp_lg(r['line'])

  for token in doc:
    if token.pos_ == 'VERB':
      lemmas.append(token.lemma_)

counter = Counter(lemmas)

most_common = counter.most_common(20)

most_common

[('ter', 11),
 ('dizer', 7),
 ('ficar', 7),
 ('fazer', 6),
 ('poder', 5),
 ('começar', 5),
 ('falar', 5),
 ('deixar', 4),
 ('anunciar', 4),
 ('acontecer', 4),
 ('disputar', 4),
 ('passar', 4),
 ('seguir', 3),
 ('esperar', 3),
 ('nascer', 3),
 ('dar', 3),
 ('parecer', 3),
 ('afirmar', 3),
 ('abrigar', 3),
 ('vir', 3)]

#### 5. Identifique e exiba bigramas do dataset cujas classes de POS sejam substantivo + adjetivo. Exemplo: equipe inglesa

In [70]:
for idx, r in texts.iterrows():

  doc = nlp_lg(r['line'])

  bigramas = []
  for token in doc:
    if token.pos_ == 'NOUN' and doc[token.i + 1].pos_ == 'ADJ':
      bigramas.append(token.text + '-' + doc[token.i + 1].text)

  texts.at[idx, 'BIGRAMAS'] = ','.join(bigramas)

texts




,index,line,filename,NOUNS,VERBS,BIGRAMAS
0,0,"O anúncio oficial deve ocorrer nesta semana, p...",ESPORTE_2005_21964.txt,"anúncio,semana,saída,contrato,chefe,escuderia","deve,ocorrer,admite,seguir,firmar,diz,ocorreu,...",anúncio-oficial
1,1,"O novo time de Massa é a Ferrari, que havia pl...",ESPORTE_2005_21964.txt,"time,Massa,contratação,evento,termos,acordo,co...","planejado,divulgar,revelaria,assinado,pode,mud...",acordo-amigável
2,2,Mas as evidências começam a aparecer. Além das...,ESPORTE_2005_21964.txt,"evidências,frases,dirigente,excitação,domingo,...","começam,aparecer,conter,estava,largar,falado",dirigente-suíço
3,3,"À tarde, deixou escapar para um jornalista ale...",ESPORTE_2005_21964.txt,"tarde,jornalista,fluência,italiano,entrevista,...","deixou,escapar,precisa,melhorar,publicada,falo...","jornalista-alemão,heptacampeão-mundial,caminho..."
4,0,"O ala-pivô Nenê, 22, maior astro do basquete n...",ESPORTE_2005_22023.txt,"ala-pivô,astro,basquete,seleção,mudanças","reiterou,defender,houver","basquete-nacional,seleção-brasileira"
...,...,...,...,...,...,...
70,3,"Se o Co-Rio não conseguir, a prefeitura arcará...",ESPORTE_2005_21873.txt,"prefeitura,obra,compromisso,exigência,construç...","conseguir,arcará,financiar","parque-aquático,complexo-multidesportivo"
71,4,"Hoje, o município já é responsável por R$ 663 ...",ESPORTE_2005_21873.txt,"município,investimento,evento",,
72,5,"O contrato, no qual a prefeitura só será respo...",ESPORTE_2005_21873.txt,"contrato,prefeitura,débito,patrocínios,consórc...","levantar,apresentado,viabilizou",
73,6,"Ruy Cézar, da secretaria municipal extraordiná...",ESPORTE_2005_21873.txt,"conta,município,prefeitura,simbologia,comitê","criada,organizar,admitiu,pode,ficar,sabe,vende...",
